# Recherche des coins de l'echequier

spécifier la camera à calibrer

In [ ]:
camera_name = "1" # nom du dossier pour camera_<nom>

In [50]:
# https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

import numpy as np
import cv2 as cv
import glob
 
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*8,3), np.float32)
objp[:,:2] = np.mgrid[0:6,0:8].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
chess_path = "camera_"+camera_name+"/chess/"
images = glob.glob('*.jpg', dir_fd=chess_path)
 
for fname in images:
 fname = dir_fd="camera_"+camera_name+"/chess/"+fname
 img = cv.imread(fname)
 gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
 # Find the chess board corners
 ret, corners = cv.findChessboardCorners(gray, (6,8), None) # (height, width), nombre de coins internes (si trop grand donne erreur)
 
 # If found, add object points, image points (after refining them)
 if ret == True:
    objpoints.append(objp)
 
 corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
 imgpoints.append(corners2)
 
 # Draw and display the corners
 cv.drawChessboardCorners(img, (7,6), corners2, ret)
 cv.imshow('img', img)
 cv.waitKey(500)
 
cv.destroyAllWindows()

# Calcul des caractéristiques internes de la camera

Les calculs sont fait à partir des points trouvés précedemment 

In [49]:
dir_path = "camera_"+camera_name+"/calibrations_matrix/"
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

np.savetxt(dir_path+'cam1_mtx.txt', mtx)
np.savetxt(dir_path+'cam1_dist.txt', dist)

# Calibration

## Toutes les images

In [79]:
uncalibrated_path = "camera_"+camera_name+"/chess/"
calibrated_path = "camera_"+camera_name+"/calibrated/"
images = glob.glob('*.jpg', dir_fd=uncalibrated_path)
 
for fname in images:
    img = cv.imread(uncalibrated_path+fname)
    h, w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    # undistort
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
    
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv.imwrite(calibrated_path +'calibrated_'+ fname, dst)

## Une image

### Importer l'image à retoucher

In [61]:
img = cv.imread('camera_1/chess/2024-06-04 11-31-32.jpg')
mtx = np.loadtxt(dir_path+'cam1_mtx.txt')
dist = np.loadtxt(dir_path+'cam1_dist.txt')
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

### Compenser la distortion

In [62]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True

# Evaluation

In [ ]:
mean_error = 0
for i in range(len(objpoints)):
 imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
 error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
 mean_error += error
 
print( "total error: {}".format(mean_error/len(objpoints)) )